In [1]:
# 通常利用torch建模需要的包
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optimizer

根据前面我们对深度学习任务的梳理，有如下几个超参数可以统一设置，方便后续调试时修改：

- batch size
- 初始学习率（初始）
- 训练次数（max_epochs）
- GPU配置

In [2]:
batch_size = 16 # batch_size
lr = 1e-4 # 学习率
max_epochs = 100 # 最大迭代轮数

In [3]:
# gpu设置

# # 方案一：使用os.environ，这种情况如果使用GPU不需要设置
# os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

# 方案二：使用“device”，后续对要使用GPU的变量用.to(device)即可
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

## 数据读入
PyTorch数据读入是通过```Dataset```+```Dataloader```的方式完成的
- ```Dataset```定义好数据的格式和数据变换形式
- ```Dataloader```用iterative（迭代器）的方式不断读入批次数据。

我们可以定义自己的Dataset类来实现灵活的数据读取，定义的类需要继承PyTorch自身的Dataset类。主要包含三个函数：

- `__init__`: 用于向类中传入外部参数，同时定义样本集
- `__getitem__`: 用于逐个读取样本集合中的元素，可以进行一定的变换，并将返回训练/验证所需的数据。iterative（迭代器）
- `__len__`: 用于返回数据集的样本数

如果是pytorch数据集```datasets```本身就有的数据，生成的模式如下：
这里使用了PyTorch自带的ImageFolder类的用于读取按一定结构存储的图片数据
- ```path```对应图片存放的目录，目录下包含若干子目录，每个子目录对应属于同一个类的图片）
- ```data_transform```可以对图像进行一定的变换，如翻转、裁剪等操作，可自己定义
```PYTHON
train_data = datasets.ImageFolder(train_path, transform=data_transform)
val_data = datasets.ImageFolder(val_path, transform=data_transform)
```

如果我们需要自定义数据集，可以参考下面模板：

In [4]:
class MyDataset(Dataset):
    def __init__(self, data_dir, info_csv, image_list, transform=None):
        """
        Args:
            data_dir: 图片的地址.
            info_csv: 包含图像索引的CSV文件的路径与相应的标签.
            image_list: 训练集和测试集文件名的txt列表地址
            transform: 在样本上应用变换
        """
        label_info = pd.read_csv(info_csv)
        image_file = open(image_list).readlines() # 读txt，按行读
        self.data_dir = data_dir
        self.image_file = image_file
        self.label_info = label_info
        self.transform = transform

    def __getitem__(self, index):
        """
        Args:传入索引
        Returns: 图像及其label
            
        """
        # strip() 方法用于移除字符串头尾指定的字符（默认为空格或换行符）或字符序列。
        image_name = self.image_file[index].strip('\n')
        
        
        raw_label = self.label_info.loc[self.label_info['Image_index'] == image_name]
        label = raw_label.iloc[:,0]
        image_name = os.path.join(self.data_dir, image_name)
        image = Image.open(image_name).convert('RGB')
        if self.transform is not None:
            image = self.transform(image)
        return image, label

    def __len__(self):
        return len(self.image_file)

构建好Dataset后，就可以使用DataLoader来按批次读入数据了，实现代码如下

In [ ]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=4, shuffle=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, num_workers=4, shuffle=False)

其中:

- batch_size：样本是按“批”读入的，batch_size就是每次读入的样本数
- num_workers：有多少个进程用于读取数据
- shuffle：是否将读入的数据打乱
- drop_last：对于样本最后一部分没有达到批次数的样本，使其不再参与训练

##  模型构建
PyTorch中神经网络构造一般是基于```Module```类的模型来完成的，它让模型构造更加灵活。

```Module```类是 nn 模块里提供的一个模型构造类，是所有神经⽹网络模块的基类，我们可以继承它来定义我们想要的模型。下面继承 Module 类构造多层感知机。

这里定义的 MLP 类重载了 Module 类的```init```函数和```forward```函数。它们分别用于创建模型参数和定义前向传播。

In [1]:
import torch
from torch import nn

class MLP(nn.Module):
  # 声明带有模型参数的层，这里声明了两个全连接层
  def __init__(self, **kwargs):
    # 调用MLP父类Block的构造函数来进行必要的初始化。这样在构造实例时还可以指定其他函数
    super(MLP, self).__init__(**kwargs)
    self.hidden = nn.Linear(784, 256) # 隐藏层参数
    self.act = nn.ReLU()              # 激活函数
    self.output = nn.Linear(256,10)   # 输出层
    
# 定义模型的前向计算，即如何根据输入x计算返回所需要的模型输出
  def forward(self, x):
    o = self.act(self.hidden(x))
    return self.output(o)   

我们已经写好一个类，所以接下来我们需要实例化这个继承了nn.Module的网络类，生成一个网络实例。

In [2]:
net = MLP()
print(net)

MLP(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


接下来我们定义一个输入X，并把它放到网络中

In [3]:
X = torch.rand(2,784)
net(X)

tensor([[-0.1236,  0.0890,  0.0968,  0.1827, -0.1538,  0.1610,  0.1313,  0.1407,
         -0.1381, -0.1908],
        [ 0.0494,  0.2237,  0.1096,  0.0623,  0.1332,  0.1412,  0.0914,  0.2664,
         -0.0886, -0.0959]], grad_fn=<AddmmBackward0>)

### 使用Module自定义层
- 不含参数的层

MyLayer 类通过继承 Module 类自定义了一个**将输入减掉均值后输出**的层，并将层的计算定义在了 forward 函数里。这个层里不含模型参数

In [4]:
import torch
from torch import nn

class MyLayer(nn.Module):
    def __init__(self, **kwargs):
        super(MyLayer, self).__init__(**kwargs)
    def forward(self, x):
        return x - x.mean()  

In [5]:
layer = MyLayer()
layer(torch.tensor([1, 2, 3, 4, 5], dtype=torch.float))

tensor([-2., -1.,  0.,  1.,  2.])

- **含模型参数的层**

我们还可以自定义含模型参数的自定义层。其中的模型参数可以通过训练学出。

**Parameter 类其实是 Tensor 的子类，如果一 个 Tensor 是 Parameter ，那么它会⾃动被添加到模型的参数列表里。**所以在⾃定义含模型参数的层时，我们应该将参数定义成 Parameter ，除了直接定义成 Parameter 类外，还可以使⽤ ParameterList 和 ParameterDict 分别定义参数的列表和字典。

In [6]:
class MyListDense(nn.Module):
    def __init__(self):
        super(MyListDense, self).__init__()
        # 每一层参数4*4，一共有三个隐藏层
        self.params = nn.ParameterList([nn.Parameter(torch.randn(4, 4)) for i in range(3)]) 
        # 增加输出层参数
        self.params.append(nn.Parameter(torch.randn(4, 1)))

    def forward(self, x):
        for i in range(len(self.params)):
            x = torch.mm(x, self.params[i])
        return x
net = MyListDense()
print(net)

MyListDense(
  (params): ParameterList(
      (0): Parameter containing: [torch.FloatTensor of size 4x4]
      (1): Parameter containing: [torch.FloatTensor of size 4x4]
      (2): Parameter containing: [torch.FloatTensor of size 4x4]
      (3): Parameter containing: [torch.FloatTensor of size 4x1]
  )
)


如果采用参数字典定义：

In [7]:
class MyDictDense(nn.Module):
    def __init__(self):
        super(MyDictDense, self).__init__()
        self.params = nn.ParameterDict({
                'linear1': nn.Parameter(torch.randn(4, 4)),
                'linear2': nn.Parameter(torch.randn(4, 1))
        })
        self.params.update({'linear3': nn.Parameter(torch.randn(4, 2))}) # 新增

    def forward(self, x, choice='linear1'):
        return torch.mm(x, self.params[choice])

net = MyDictDense()
print(net)

MyDictDense(
  (params): ParameterDict(
      (linear1): Parameter containing: [torch.FloatTensor of size 4x4]
      (linear2): Parameter containing: [torch.FloatTensor of size 4x1]
      (linear3): Parameter containing: [torch.FloatTensor of size 4x2]
  )
)


## 损失函数
### 二分类交叉熵损失函数
$$\ell(x, y)=\left\{\begin{array}{ll}
\operatorname{mean}(L), & \text { if reduction }=\text { 'mean' } \\
\operatorname{sum}(L), & \text { if reduction }=\text { 'sum' }
\end{array}\right.$$

In [9]:
torch.nn.BCELoss(weight=None,  # `weight`:每个类别的loss设置权值
                 size_average=None, # 不建议采用，使用reduction.`size_average`:数据为bool，为True时，返回的loss为平均值；为False时，返回的各样本的loss之和。
                 reduce=None, # `reduce`:数据类型为bool，为True时，loss的返回是标量。
                 reduction='mean')

BCELoss()

In [10]:
m = nn.Sigmoid()
loss = nn.BCELoss()
input = torch.randn(3, requires_grad=True)
target = torch.empty(3).random_(2)
output = loss(m(input), target)
output.backward()

In [11]:
print('BCELoss损失函数的计算结果为',output)

BCELoss损失函数的计算结果为 tensor(1.3421, grad_fn=<BinaryCrossEntropyBackward0>)


### 交叉熵损失

$$\operatorname{loss}(x, \text { class })=-\log \left(\frac{\exp (x[\text { class }])}{\sum_{j} \exp (x[j])}\right)=-x[\text { class }]+\log \left(\sum_{j} \exp (x[j])\right)$$ 

In [ ]:
torch.nn.CrossEntropyLoss(weight=None, # `weight`:每个类别的loss设置权值。
                          size_average=None, # 不建议采用，使用reduction.`size_average`:数据为bool，为True时，返回的loss为平均值；为False时，返回的各样本的loss之和。
                          ignore_index=-100, # 忽略某个类的损失函数。
                          reduce=None, # 不建议采用，使用reduction.数据类型为bool，为True时，loss的返回是标量
                          reduction='mean') 

In [14]:
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)
output.backward()
output

tensor(1.8728, grad_fn=<NllLossBackward0>)

[主要的已经构建好的损失函数参数解释](https://github.com/datawhalechina/thorough-pytorch/blob/main/%E7%AC%AC%E4%B8%89%E7%AB%A0%20PyTorch%E7%9A%84%E4%B8%BB%E8%A6%81%E7%BB%84%E6%88%90%E6%A8%A1%E5%9D%97/3.5%20%E6%8D%9F%E5%A4%B1%E5%87%BD%E6%95%B0.md)

## 优化器
优化器本质上就是解方程的工具。对于一个网络，我们想要找到最优的模型参数，以经典的resnet-50为例，它大约有2000万个系数需要进行计算，如果我们直接暴力穷举一遍参数，这种方法实施可能性基本为0，堪比愚公移山plus的难度。所以工业界一般采用BP+优化器逼近求解。

优化器是根据网络反向传播的梯度信息来更新网络的参数，以起到降低loss函数计算值，使得模型输出更加接近真实标签。

Pytorch很人性化的给我们提供了一个优化器的库torch.optim，在这里面提供了十种优化器。
+ torch.optim.ASGD
+ torch.optim.Adadelta
+ torch.optim.Adagrad
+ torch.optim.Adam
+ torch.optim.AdamW
+ torch.optim.Adamax
+ torch.optim.LBFGS
+ torch.optim.RMSprop
+ torch.optim.Rprop
+ torch.optim.SGD
+ torch.optim.SparseAdam

而以上这些优化算法均继承于`Optimizer`，下面我们先来看下所有优化器的基类`Optimizer`。定义如下：

In [16]:
class Optimizer(object):
    def __init__(self, params, defaults):        
        self.defaults = defaults
        self.state = defaultdict(dict)
        self.param_groups = []

**`Optimizer`有三个属性：**

+ `defaults`：存储的是优化器的超参数

In [17]:
{'lr': 0.1, # 学习率
 'momentum': 0.9,  # momentum 角动量
 'dampening': 0, # 阻尼系数
 'weight_decay': 0, # 权值衰减
 'nesterov': False} #  Nesterov 加速

{'lr': 0.1,
 'momentum': 0.9,
 'dampening': 0,
 'weight_decay': 0,
 'nesterov': False}

`state`：参数的缓存

```PYTHON
defaultdict(<class 'dict'>, 
            {tensor([[ 0.3864, -0.0131],
                    [-0.1911, -0.4511]], requires_grad=True): {'momentum_buffer': tensor([[0.0052, 0.0052],
                                                                                 [0.0052, 0.0052]])}})
```

`param_groups`：管理的参数组，是一个list，其中每个元素是一个字典，顺序是params，lr，momentum，dampening，weight_decay，nesterov，

```PYTHON
[{'params': [tensor([[-0.1022, -1.6890],
                     [-1.5116, -1.7846]], requires_grad=True)], 
  'lr': 1, 
  'momentum': 0, 
  'dampening': 0, 
  'weight_decay': 0, 
  'nesterov': False}]
```

**`Optimizer`还有以下的方法：**

- `zero_grad()`：清空所管理参数的梯度，PyTorch的特性是张量的梯度不自动清零，因此每次反向传播后都需要清空梯度。

In [22]:
def zero_grad(self, set_to_none: bool = False):
    for group in self.param_groups: # 遍历每一个网络层的参数tensor
        for p in group['params']: 
            if p.grad is not None:  #梯度不为空
                if set_to_none: 
                    p.grad = None
                else:
                    if p.grad.grad_fn is not None:
                        p.grad. detach_() # 从当前计算图剪下，后续不能利用梯度
                    else:
                        p.grad.requires_grad_(False)
                    p.grad.zero_()# 梯度设置为0

- `step()`：执行一步梯度更新，参数更新

In [23]:
def step(self, closure): 
    raise NotImplementedError

- `add_param_group()`：添加参数组
- `load_state_dict()` ：加载状态参数字典，可以用来进行模型的断点续训练，继续上次的参数进行训练
- `state_dict()`：获取优化器当前状态信息字典

### 实际操作

In [24]:
import os
import torch

# 设置权重，服从正态分布  --> 2 x 2
weight = torch.randn((2, 2), requires_grad=True)

# 设置梯度为全1矩阵  --> 2 x 2
weight.grad = torch.ones((2, 2))

# 输出现有的weight和data
print("The data of weight before step:\n{}".format(weight.data))
print("The grad of weight before step:\n{}".format(weight.grad))

The data of weight before step:
tensor([[ 0.2549,  0.0609],
        [-2.0199, -0.8439]])
The grad of weight before step:
tensor([[1., 1.],
        [1., 1.]])


In [25]:
# 实例化优化器
optimizer = torch.optim.SGD([weight], lr=0.1, momentum=0.9)
# 进行一步操作
optimizer.step()
# 查看进行一步后的值，梯度
print("The data of weight after step:\n{}".format(weight.data))
print("The grad of weight after step:\n{}".format(weight.grad))

The data of weight after step:
tensor([[ 0.1549, -0.0391],
        [-2.1199, -0.9439]])
The grad of weight after step:
tensor([[1., 1.],
        [1., 1.]])


In [26]:
# 权重清零
optimizer.zero_grad()
# 检验权重是否为0
print("The grad of weight after optimizer.zero_grad():\n{}".format(weight.grad))
# 输出参数
print("optimizer.params_group is \n{}".format(optimizer.param_groups))
# 查看参数位置，optimizer和weight的位置一样，我觉得这里可以参考Python是基于值管理
print("weight in optimizer:{}\nweight in weight:{}\n".format(id(optimizer.param_groups[0]['params'][0]), id(weight)))

The grad of weight after optimizer.zero_grad():
tensor([[0., 0.],
        [0., 0.]])
optimizer.params_group is 
[{'params': [tensor([[ 0.1549, -0.0391],
        [-2.1199, -0.9439]], requires_grad=True)], 'lr': 0.1, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'nesterov': False}]
weight in optimizer:2243298833568
weight in weight:2243298833568



In [27]:
# 添加参数：weight2
weight2 = torch.randn((3, 3), requires_grad=True)
optimizer.add_param_group({"params": weight2, 'lr': 0.0001, 'nesterov': True})
# 查看现有的参数
print("optimizer.param_groups is\n{}".format(optimizer.param_groups))
# 查看当前状态信息
opt_state_dict = optimizer.state_dict()
print("state_dict before step:\n", opt_state_dict)

optimizer.param_groups is
[{'params': [tensor([[ 0.1549, -0.0391],
        [-2.1199, -0.9439]], requires_grad=True)], 'lr': 0.1, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'nesterov': False}, {'params': [tensor([[ 0.2742, -0.2071, -1.3786],
        [-1.3224,  1.9979, -0.0408],
        [ 1.1698,  0.6113,  0.3265]], requires_grad=True)], 'lr': 0.0001, 'nesterov': True, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0}]
state_dict before step:
 {'state': {0: {'momentum_buffer': tensor([[1., 1.],
        [1., 1.]])}}, 'param_groups': [{'lr': 0.1, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [0]}, {'lr': 0.0001, 'nesterov': True, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'params': [1]}]}


In [30]:
# 进行50次step操作
for _ in range(50):
    optimizer.step()

# 输出现有状态信息
print("state_dict after step:\n", optimizer.state_dict())
# # 保存参数信息
# torch.save(optimizer.state_dict(),os.path.join(r"D:\pythonProject\Attention_Unet", "optimizer_state_dict.pkl"))
# print("----------done-----------")
# # 加载参数信息
# state_dict = torch.load(r"D:\pythonProject\Attention_Unet\optimizer_state_dict.pkl") # 需要修改为你自己的路径
# optimizer.load_state_dict(state_dict)
# print("load state_dict successfully\n{}".format(state_dict))

# 输出最后属性信息
print("\n{}".format(optimizer.defaults))
print("\n{}".format(optimizer.state))
#print("\n{}".format(optimhttps://segmentfault.com/izer.param_groups))

state_dict after step:
 {'state': {0: {'momentum_buffer': tensor([[0.0052, 0.0052],
        [0.0052, 0.0052]])}}, 'param_groups': [{'lr': 0.1, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [0]}, {'lr': 0.0001, 'nesterov': True, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'params': [1]}]}

{'lr': 0.1, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'nesterov': False}

defaultdict(<class 'dict'>, {tensor([[-0.7404, -0.9345],
        [-3.0153, -1.8393]], requires_grad=True): {'momentum_buffer': tensor([[0.0052, 0.0052],
        [0.0052, 0.0052]])}})


##### 注意：

1. 每个优化器都是一个类，我们一定要进行实例化才能使用，比如下方实现：

In [ ]:
class Net(nn.Moddule):
    ···
net = Net()
optim = torch.optim.SGD(net.parameters(),lr=lr)
optim.step()

2. optimizer在一个神经网络的epoch中需要实现下面两个步骤：

    1. 梯度置零
    2. 梯度更新

In [ ]:
optimizer = torch.optim.SGD(net.parameters(), lr=1e-5)
for epoch in range(EPOCH):
	...
	optimizer.zero_grad()  #梯度置零
	loss = ...             #计算loss
	loss.backward()        #BP反向传播
	optimizer.step()       #梯度更新

## 训练与评估 流程化

首先应该设置模型的状态：如果是训练状态，那么模型的参数应该支持反向传播的修改；如果是验证/测试状态，则不应该修改模型参数。

In [ ]:
model.trainbin  # 训练状态
model.eval()   # 验证/测试状态

我们前面在DataLoader构建完成后介绍了如何从中读取数据，在训练过程中使用类似的操作即可，区别在于此时要用for循环读取DataLoader中的全部数据。

In [ ]:
for data, label in train_loader:

之后将数据放到GPU上用于后续计算，此处以.cuda()为例

In [ ]:
data, label = data.cuda(), label.cuda()

开始用当前批次数据做训练时，应当先将优化器的梯度置零：

In [ ]:
optimizer.zero_grad()

之后将data送入模型中训练：

In [ ]:
output = model(data)

根据预先定义的criterion计算损失函数：

In [ ]:
loss = criterion(output, label)

将loss反向传播回网络：

In [ ]:
loss.backward()

使用优化器更新模型参数：

In [ ]:
optimizer.step()

验证/测试的流程基本与训练过程一致，不同点在于：

- 需要预先设置torch.no_grad，以及将model调至eval模式
- 不需要将优化器的梯度置零
- 不需要将loss反向回传到网络
- 不需要更新optimizer

一个完整的训练过程如下所示：

In [ ]:
def train(epoch):
    model.train() # 设置模型模式
    train_loss = 0 # 初始化训练损失
    for data, label in train_loader: # 每次批量读取数据
        data, label = data.cuda(), label.cuda() # 转换成gpu模式
        optimizer.zero_grad() # 优化器梯度清零
        output = model(data) # 获取模型输出
        loss = criterion(label, output) # 求出损失
        loss.backward() # 反向传播损失
        optimizer.step() # 更新模型参数
        train_loss += loss.item()*data.size(0) # 累计损失
    train_loss = train_loss/len(train_loader.dataset) # 本次Epoch平均损失
		print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch, train_loss))

对应的，一个完整的验证过程如下所示：

In [ ]:
def val(epoch):       
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for data, label in val_loader:
            data, label = data.cuda(), label.cuda()
            output = model(data)
            preds = torch.argmax(output, 1)
            loss = criterion(output, label)
            val_loss += loss.item()*data.size(0)
            running_accu += torch.sum(preds == label.data)
    val_loss = val_loss/len(val_loader.dataset)
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch, val_loss))